In [131]:
from transformers import AutoTokenizer
import numpy as np
import json
import re
from difflib import SequenceMatcher

In [23]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-large")

c:\Users\fardin\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [26]:
id_vocab = {v:k for k,v in tokenizer.vocab.items()}
all_vocab_str = []
for i in range(len(id_vocab)):
    all_vocab_str.append(id_vocab[i])

In [94]:
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob

all_vocab_polarity = []
all_vocab_subjectivity = []
nlp = spacy.load('en_core_web_sm', disable=['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner'])
nlp.add_pipe('spacytextblob')
for i in range(len(all_vocab_str)):
    text = all_vocab_str[i]
    doc = nlp(text)
    polarity = 0
    subjectivity = 0
    for t in doc:
        polarity+=t._.blob.polarity
        subjectivity+=t._.blob.subjectivity
    all_vocab_polarity.append(polarity)
    all_vocab_subjectivity.append(subjectivity)
all_vocab_polarity = np.array(all_vocab_polarity)
all_vocab_subjectivity = np.array(all_vocab_subjectivity)

In [231]:
temp_str = """ the texts: """
def create_batch_request_for_openai(tokens_list):
    batch_line = temp_str
    batch_line += tokens_list[0]
    for t in tokens_list[1:]:
        batch_line += ' | '
        batch_line += t
    return batch_line
lines = {}
interval = 50
for i in range(0, len(all_vocab_str), interval):
    upper_bound = min(i+interval, len(all_vocab_str))
    lines[(i, upper_bound)] = create_batch_request_for_openai(all_vocab_str[i:upper_bound])

In [232]:
next(iter(lines))

(0, 50)

In [233]:
(len(all_vocab_str)/500)

256.002

In [240]:
system_content = """As sentiment classifier, write polarity and subjectivity for the provided texts. For example for the given text 'good' write 'good: polarity=0.7, subjectivity=0.6' and for the text 'bad' write 'bad: polarity=-0.7, subjectivity=0.67' and for the given text 'good | bad' write 'good: polarity=0.7, subjectivity=0.6 | bad: polarity=-0.7, subjectivity=0.67'. I will give you exactly {t_count} texts and you just write the polarity and sensitivity without any other words for exact same {t_count} texts."""

"As sentiment classifier, write polarity and subjectivity for the provided texts. For example for the given text 'good' write 'good: polarity=0.7, subjectivity=0.6' and for the text 'bad' write 'bad: polarity=-0.7, subjectivity=0.67' and for the given text 'good | bad' write 'good: polarity=0.7, subjectivity=0.6 | bad: polarity=-0.7, subjectivity=0.67'. I will give you exactly 50 texts and you just write the polarity and sensitivity without any other words for exact same 50 texts."

In [243]:
jsonl_lines = []
for k, v in lines.items():
    # print(k, len(v.split('|')))
    system_content = """As sentiment classifier, write polarity and subjectivity for the provided texts. For example for the given text 'good' write 'good: polarity=0.7, subjectivity=0.6' and for the text 'bad' write 'bad: polarity=-0.7, subjectivity=0.67' and for the given text 'good | bad' write 'good: polarity=0.7, subjectivity=0.6 | bad: polarity=-0.7, subjectivity=0.67'. I will give you exactly {t_count} texts and you just write the polarity and sensitivity without any other words for exact same {t_count} texts."""
    jsonl_lines.append({"custom_id": f"{k}", "method": "POST", "url": "/v1/chat/completions", "body": {"model": "gpt-4o-mini", "messages": [{"role": "system", "content": system_content.format(t_count=len(v.split('|')))},{"role": "user", "content": v}],"max_tokens": 1000}})


In [244]:
# jsonl_lines2 = []
# for k, v in lines.items():
#     jsonl_lines2.append(system_content + v)
    # jsonl_lines2.append(str.join(system_content, v))

In [245]:
with open(r'DataManipulation\tokens_polarity.jsonl', 'w', encoding="utf-8") as f:
    for i in range(len(jsonl_lines)):
        f.write(json.dumps(jsonl_lines[i])+'\n')

In [209]:
with open(r'DataManipulation\tokens_polarity3.jsonl', 'w', encoding="utf-8") as f:
    for i in range(len(jsonl_lines2)):
        f.write(json.dumps(jsonl_lines2[i])+'\n')

In [195]:
with open(r'DataManipulation\tokens_polarity2.jsonl', 'w', encoding="utf-8") as f:
    for i in range(len(jsonl_lines2)):
        f.write(json.dumps(jsonl_lines2[i])+'\n')
        break

In [247]:
with open(r'DataManipulation\token_polarity_mini_2_batch_674650e577508190b2e501b187ce6ed7_output.jsonl', 'r') as f:
    all_lines = []
    line_stream = f.readlines()
    for line in line_stream:
        all_lines.append(json.loads(line))

In [254]:
content = all_lines[51]['response']['body']['choices'][0]['message']['content']
content = str.split(content, '|')
len(content)

49

In [271]:
' : : polarity=0.0, subjectivity=0.0 '.split(':')

[' ', ' ', ' polarity=0.0, subjectivity=0.0 ']

In [276]:
from copy import copy

def get_string_ints(string_sample):
    str_arr = re.findall(r'\d+', string_sample)
    int_arr = [int(s) for s in str_arr]
    return int_arr

def get_string_floats(string_sample):
    all_floats = re.findall(r"[-+]?(?:\d*\.*\d+)", string_sample)
    all_floats = [float(f) for f in all_floats]
    return all_floats

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

def map_content_to_tokens(content, tokens):
    
    content_dict = dict()
    t_c_dict = dict()
    content_keys = []
    for c in content:
        c = c.split(':')
        if len(c) == 3:
            c= [':', c[2]]
        elif len(c) ==0:
            continue
        elif len(c)==1 or len(c) >3:
            print(f"failed: {c}")
            continue
        # print(f'cccc: {c}')
        content_dict[c[0]] = c[1]
        content_keys.append(c[0])
    tokens_ = copy(tokens)
    for c in content_keys:
        most_similar_token = tokens_[0]
        base_sim = similar(c, most_similar_token)
        for t in tokens_[1:]:
            new_sim = similar(c, t)
            if new_sim > base_sim:
                base_sim = new_sim
                most_similar_token = t
        t_c_dict[most_similar_token] = c
        tokens_.remove(most_similar_token)
    return t_c_dict, content_dict

all_vocab_sentiments = dict()
k = 0
for j in range(len(all_lines)):
    ids_range = get_string_ints(all_lines[j]['custom_id'])
    content = all_lines[j]['response']['body']['choices'][0]['message']['content']
    content = str.split(content, '|')
    # print(ids_range)
    # print(len(content), content)
    list_to_remove = []
    for i in range(len(content)):
        if content[i].strip() =='':
            list_to_remove.append(i)
    for ltr in list_to_remove:
        del content[ltr]
    t_c_dict, content_dict = map_content_to_tokens(content, all_vocab_str[ids_range[0]: ids_range[1]])
            
    for i in range(ids_range[0], ids_range[1]):
        
        if all_vocab_str[i] not in t_c_dict:
            k += 1
            # print(k, 'failed', i, t, all_vocab_str[i])
            all_vocab_sentiments[all_vocab_str[i]] = [0, 0]
            continue
        
        # print(i, all_vocab_str[i], t_c_dict[all_vocab_str[i]], content_dict[t_c_dict[all_vocab_str[i]]])
        content_info = content_dict[t_c_dict[all_vocab_str[i]]].split(',')
        content_info = [t.strip() for t in content_info]
        content_info = sorted(content_info)
        polarity = get_string_floats(content_info[0])
        # print(content_info)
        subjectivity = get_string_floats(content_info[1])
        all_vocab_sentiments[all_vocab_str[i]]  = [polarity, subjectivity]
    

failed: [' ']
failed: ['ATFORM ']


In [292]:
vocab_sentiments = []
for i in range(len(all_vocab_str)):
    vocab_sentiments.append(np.array(all_vocab_sentiments[all_vocab_str[i]]).squeeze())
vocab_sentiments = np.vstack(vocab_sentiments)

397


In [293]:
vocab_sentiments.shape

(128001, 2)

In [300]:
for i in range(len(all_vocab_str)):
    if 'terrible' in all_vocab_str[i]:
        print(i, vocab_sentiments[i])
        break

4838 [-0.7   0.67]


In [302]:
np.save(r'Data\ReducedEmbeddings\polarity_debertav3_tokens_gpt_mini_emb.npy', vocab_sentiments, allow_pickle=False)

In [291]:
np.vstack(vocab_sentiments).shape

(128001, 2)

In [110]:
all_lines[0]['response']['body']['choices'][0]['message']['content']

{'id': 'batch_req_67461a2c0660819092aed4151fd0baaf',
 'custom_id': '(0, 50)',
 'response': {'status_code': 200,
  'request_id': 'a2db5f65ff33cb9c21142e0ebc71b861',
  'body': {'id': 'chatcmpl-AXv2UjwijaVpPfAhw9CXFPjLbgBoN',
   'object': 'chat.completion',
   'created': 1732647446,
   'model': 'gpt-4o-2024-08-06',
   'choices': [{'index': 0,
     'message': {'role': 'assistant',
      'content': '[PAD]: polarity=0.0, subjectivity=0.0 | [CLS]: polarity=0.0, subjectivity=0.0 | [SEP]: polarity=0.0, subjectivity=0.0 | [UNK]: polarity=0.0, subjectivity=0.0 | <0x00>: polarity=0.0, subjectivity=0.0 | <0x01>: polarity=0.0, subjectivity=0.0 | <0x02>: polarity=0.0, subjectivity=0.0 | <0x03>: polarity=0.0, subjectivity=0.0 | <0x04>: polarity=0.0, subjectivity=0.0 | <0x05>: polarity=0.0, subjectivity=0.0 | <0x06>: polarity=0.0, subjectivity=0.0 | <0x07>: polarity=0.0, subjectivity=0.0 | <0x08>: polarity=0.0, subjectivity=0.0 | <0x09>: polarity=0.0, subjectivity=0.0 | <0x0A>: polarity=0.0, subjectivi